教程地址 https://conda.io/projects/conda/en/latest/

# Concepts
+ Conda commands
+ Conda packages
+ Conda package specification
+ Conda channels
+ Conda environments
+ Installing with conda
+ Conda performance
+ Conda for data scientists

## Conda commands
作用：
+ conda命令可以查询并搜索Anaconda包索引和当前Anaconda安装
+ 创建新的conda环境
+ 在现有的conda环境中安装和更新包  

> Tips:  
> You can abbreviate many frequently used command options that are preceded by 2 dashes (--) to just 1 dash and the first letter of the option. So --name and -n are the same, and --envs and -e are the same.

command reference https://conda.io/projects/conda/en/latest/commands.html  
Viewing command-line help https://conda.io/projects/conda/en/latest/user-guide/tasks/view-command-line-help.html 

## Conda packages
### What is a conda package?
A conda package is a compressed tarball file (.tar.bz2) or .conda file that contains:
+ system-level libraries  
+ Python or other modules  
+ executable programs and other components  
+ metadata under the info/ directory  
+ a collection of files that are installed directly into an install prefix

### .conda file format
These compressed files can be significantly smaller than their bzip2 equivalents. In addition, they decompress much more quickly. .conda is the preferred file format to use where available, although we continue to provide .tar.bz2 files in tandem.
> Note  
> In conda 4.7 and later, you cannot use package names that end in “.conda” as they conflict with the .conda file format for packages.

### Using packages
+ You may search for packages
      conda search scipy
+ You may install a packages
      conda install scipy
+ You may build a package after installing conda build
      conda build my_fun_package
   

### Package structure
    .
    ├── bin
    │   └── pyflakes
    ├── info
    │   ├── LICENSE.txt
    │   ├── files
    │   ├── index.json
    │   ├── paths.json
    │   └── recipe
    └── lib
        └── python3.5

+ bin contains relevant binaries for the package
+ lib contains the relevant library files (eg. the .py files)
+ info contains package metadata

### Metapackages
当conda包仅用于元数据不含任何文件时，它被称为元包  
An example of a metapackage is "anaconda," which collects together all the packages in the Anaconda installer. The command conda create -n envname anaconda creates an environment that exactly matches what would be created from the Anaconda installer. You can create metapackages with the conda metapackage command. Include the name and version in the command.  
+ Anaconda metapackage  
The Anaconda metapackage is used in the creation of the Anaconda Distribution installers so that they have a set of packages associated with them.  
read more https://www.anaconda.com/whats-in-a-name-clarifying-the-anaconda-metapackage/
+ Mutex metapackage  
    A mutex metapackage is a very simple package that has a name.  
    Let's look at some examples for how to use mutex metapackages to build NumPy against different BLAS implementations.
    + Building NumPy with BLAS variants  
        If you build NumPy with MKL, you also need to build SciPy, scikit-learn, and anything else using BLAS also with MKL. It is important to ensure that these “variants” (packages built with a particular set of options) are installed together and never with an alternate BLAS implementation.
    + Installing NumPy with BLAS variants
        To specify which variant of NumPy that you want, you could potentially specify the BLAS library you want:
          conda install numpy mkl
        This pathway may lead to some ambiguity and solutions with mixed BLAS, so using the metapackage is recommended. To specify MKL-powered NumPy in a non-ambiguous way, you can specify the mutex package (either directly or indirectly):
          conda install numpy “blas=*=mkl”
    + Track_features  
        One of conda’s optimization goals is to minimize the number of track_features needed to specify the desired specs. By adding track_features to one or more of the options, conda will de-prioritize it or “weigh it down.” The lowest priority package is the one that would cause the most track_features to be activated in the environment. The default package among many variants is the one that would cause the least track_features to be activated.
    + More info  
        https://github.com/AnacondaRecipes/aggregate/blob/9635228/vs2017/meta.yaml#L24

### Noarch packages
Noarch packages are packages that are not architecture specific and therefore only have to be built once. Noarch packages are either generic or Python. Noarch generic packages allow users to distribute docs, datasets, and source code in conda packages  
#### Noarch Python
Noarch Python packages cut down on the overhead of building multiple different pure Python packages on different architectures and Python versions by sorting out platform and Python version-specific differences at install time.  
In order to qualify as a noarch Python package, all of the following criteria must be fulfilled:
+ No compiled extensions
+ No post-link or pre-link or pre-unlink scripts
+ No OS-specific build scripts
+ No python version specific requirements
+ No skips except for Python version. If the recipe is py3 only, remove skip statement and add version constraint on Python in host and run section.
+ 2to3 is not used
+ Scripts argument in setup.py is not used
+ If console_script entrypoints are in setup.py, they are listed in meta.yaml
+ No activate scripts
+ Not a dependency of conda

### Link and unlink scripts
You may optionally execute scripts before and after the link and unlink steps.  
### More information
Go deeper on how to manage packages. Learn more about package metadata, repository structure and index, and package match specifications at Package specifications.

## Conda package specification
### Package metadata
The info/ directory contains all metadata about a package. Files in this location are not installed under the install prefix. Although you are free to add any file to this directory, conda only inspects the content of the files discussed below.
+ Info
    + files  
      a list of all the files in the package (not included in info/)        
    + index.json  
      metadata about the package including platform, version, dependencies, and build info  
    + paths.json  
        a list of files in the package, along with their associated SHA-256, size in bytes, and the type of path (eg. hardlink vs. softlink)
+ info/index.json  
    This file contains basic information about the package, such as name, version, build string, and dependencies.
+ info/files  
    Lists all files that are part of the package itself, 1 per line. 
+ info/has_prefix  
    Optional file.Lists all files that contain a hard-coded build prefix or placeholder prefix, which needs to be replaced by the install prefix at installation time.
+ info/license.txt  
    Optional file. The software license for the package.
+ info/no_link  
    Optional file. Lists all files that cannot be linked - either soft-linked or hard-linked - into environments and are copied instead.
+ info/about.json  
    Optional file. Contains the entries in the about section of the meta.yaml file.
+ info/recipe  
    A directory containing the full contents of the build recipe.
+ meta.yaml.rendered  
    The fully rendered build recipe. See conda render.

### Repository structure and index
A conda repository - or channel - is a directory tree, usually served over HTTPS, which has platform subdirectories, each of which contain conda packages and a repository index. 
### Package match specifications
This match specification is not the same as the syntax used at the command line with conda install, such as conda install python=3.4. Internally, conda translates the command line syntax to the spec defined in this section.  
EXAMPLE: python=3.4 is translated to python 3.4\*.  
When using the command line, put double quotes around any package version specification that contains the space character or any of the following characters: <, >, \*, or /  
EXAMPLE:
>conda install numpy=1.11  
conda install numpy==1.11  
conda install "numpy>1.11"  
conda install "numpy=1.11.1|1.11.3"  
conda install "numpy>=1.8,<2"
### Version ordering
类VersionOrder（对象）实现版本字符串之间的顺序关系  
版本字符串可以包含常用的字母数字字符（A-Za-z0-9），通过点和下划线分隔成组件。不允许空白段（即两个连续的点、前导/尾随下划线）  
__支持的版本字符串__2  
Conda supports six types of version strings:
+ Release versions contain only integers, e.g. 1.0, 2.3.5.
+ Pre-release versions use additional letters such as a or rc, for example 1.0a1, 1.2.beta3, 2.3.5rc3.
+ Development versions are indicated by the string dev, for example 1.0dev42, 2.3.5.dev12.
+ Post-release versions are indicated by the string post, for example 1.0post1, 2.3.5.post2.
+ Tagged versions have a suffix that specifies a particular property of interest, e.g. 1.1.parallel. Tags can be added to any of the preceding 4 types. As far as sorting is concerned, tags are treated like strings in pre-release versions.
+ An optional local version string separated by + can be appended to the main (upstream) version string. It is only considered in comparisons when the main versions are equal, but otherwise handled in exactly the same manner.

__可预测的版本排序__  
为了获得可预测的版本顺序，关键是要保持给定包的版本号方案随时间的推移保持一致。Conda将预发布版本视为低于发布版本。

## Conda channels
### What is a "conda channel"?
Conda通道是存储包的位置。它们是托管和管理包的基础。Conda包是从远程通道下载的，这些通道是指向包含Conda包的目录的url。conda命令搜索一组默认频道，包将自动从https://repo.anaconda.com/pkgs/ 下载和更新。您可以修改自动搜索的远程频道。详见 https://conda.io/projects/conda/en/latest/user-guide/configuration/use-condarc.html#config-channels
### Specifying channels when installing packages
From the command line use --channel
> conda install scipy --channel conda-forge

You may specify multiple channels by passing the argument multiple times:
> conda install scipy --channel conda-forge --channel bioconda

优先级从左到右递减-第一个参数的优先级高于第二个参数  
From the command line use --override-channels 只搜索指定的通道，而不是在.condarc中配置的任何通道。这也会忽略conda的默认频道
> conda search scipy --channel file:/<path to\>/local-channel --override-channels
### Conda clone channel RSS feed
我们提供了一个RSS提要，它表示频道克隆所克隆的所有内容，现在可以在CDN（内容交付网络）后面使用。RSS提要显示在滚动的两周时间内发生的事情，对于查看包的位置或是否运行了同步非常有用。

## Conda environments
conda环境是包含已安装的conda包的特定集合的目录。例如，您可能有一个使用NumPy 1.7及其依赖项的环境，以及另一个使用numpy1.6进行遗留测试的环境。如果更改一个环境，则不会影响其他环境。您可以很容易地激活或停用环境，这就是您在它们之间切换的方式。您还可以通过向某人提供environment.yaml文件的副本来与他们共享您的环境。详见
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

# Tasks

## Managing environments
使用conda，您可以创建、导出、列出、删除和更新安装了不同版本Python和/或包的环境。在环境之间切换或移动称为激活环境。也可以共享环境文件。
### Creating an environment with commands
默认情况下，环境安装在conda目录的envs目录中。有关指定其他路径的信息，请参阅Specifying a location for an environment 或运行conda create--help。  
Use the terminal or an Anaconda Prompt for the following steps:
1. To create an environment:
> conda create --name myenv
>> Note  
Replace myenv with the environment name.
2. When conda asks you to proceed, type y:
> proceed ([y]/n)?

    这将在/envs/中创建myenv环境。此环境使用的Python版本与您当前使用的Python版本相同，因为您没有指定版本。
3. To create an environment with a specific version of Python:
> conda create -n myenv python=3.6

4. To create an environment with a specific package:
> conda create -n myenv scipy  

    OR
> conda create -n myenv python  
conda install -n myenv scipy

5. To create an environment with a specific version of a package:
> conda create -n myenv scipy=0.15.0
    
    OR
> conda create -n myenv python  
conda install -n myenv scipy=0.15.0

6. To create an environment with a specific version of Python and multiple packages:
> conda create -n myenv python=3.6 scipy=0.15.0 astroid babel

要在每次创建新环境时自动安装pip或其他程序，请将默认程序添加到.condarc配置文件的“创建默认软件包”部分。每次创建新环境时都会安装默认包。如果不希望在特定环境中安装默认软件包，请使用--no default packages标志：
> conda create --no-default-packages -n myenv python

### Creating an environment from an environment.yml file
Use the terminal or an Anaconda Prompt for the following steps:
1. Create the environment from the environment.yml file:
> conda env create -f environment.yml

yml文件的第一行设置新环境的名称。有关详细信息，请参见Creating an environment file manually。
2. Activate the new environment: 
> conda activate myenv
3. Verify that the new environment was installed correctly:
> conda env list

    You can also use 
> conda info --envs

### Specifying a location for an environment
通过在创建环境时提供目标目录的路径，可以控制conda环境的位置。例如，以下命令将在当前工作目录的子目录envs中创建新环境：
> conda create --prefix ./envs jupyterlab=0.35 matplotlib=3.1 numpy=1.16

然后，使用用于激活按名称创建的环境的相同命令激活以前缀创建的环境：
> conda activate ./envs

在创建环境时指定项目目录的子目录的路径有以下好处：
+ 通过将环境包含为子目录，可以很容易地判断项目是否使用独立环境。
+ 它使您的项目更加独立，因为所有内容（包括所需的软件）都包含在单个项目目录中。
+ 可以对所有环境使用相同的名称。如果将所有环境保存在envs文件夹中，则必须为每个环境指定不同的名称。

将conda环境放置在默认envs文件夹之外时，需要注意一些事项:
+ Conda无法再使用--name标志找到您的环境。通常需要将--prefix标志与环境的完整路径一起传递才能找到环境。
+ 在创建conda环境时指定安装路径可以使命令提示符以活动环境的绝对路径而不是环境名称作为前缀。

使用前缀激活环境后，提示将类似于以下内容：
> (/Users/USER_NAME/research/data-science/PROJECT_NAME/envs) $

To remove this long prefix in your shell prompt, modify the env_prompt setting in your .condarc file:
> conda config --set env_prompt '({name})'

This will edit your .condarc file if you already have one or create a .condarc file if you do not.  
现在，命令提示符将显示活动环境的通用名称，即环境根文件夹的名称：
> cd project-directory  
> conda activate ./env  
(env) project-directory $